In [1]:
import pandas as pd
import numpy as np
import gcsfs
import subprocess

In [94]:
list_files = subprocess.run(["gsutil", "ls", "-l", "gs://wagon-data-802-marchand/twitter_data/"], stdout=subprocess.PIPE, text=True, check=True)
array_of_files = np.array([file.strip().split() for file in list_files.stdout.split('\n')[:-2]])
last_file = array_of_files[array_of_files[:, 1].argsort()][-1,2]


fs = gcsfs.GCSFileSystem(project='wagon-bootcamp-802')
with fs.open(last_file) as f:
    df = pd.read_csv(f)

In [4]:
df = pd.read_csv("../twitter_data_clean_data.csv")

In [5]:
df

,text,created_at,id,lang
0,📸 paddock today 😊 height difference though 👀 f...,2022-07-21 13:49:48+00:00,1550115832595824641,en
1,mercedes team boss toto wolff says “ everybody...,2022-07-21 13:49:36+00:00,1550115781257572353,en
2,racing news alonso confident french gp im phys...,2022-07-21 13:49:35+00:00,1550115777251926017,en
3,📰 haas use new front wing sideplate weekends f...,2022-07-21 13:49:35+00:00,1550115776538951680,en
4,’ bad luck last races competitive updates week...,2022-07-21 13:49:29+00:00,1550115750483935234,en
...,...,...,...,...
61,full confidence team team come back made mista...,2022-07-21 13:45:21+00:00,1550114712892743681,en
62,f sebastian vettel rumor hes moving mclaren an...,2022-07-21 13:45:17+00:00,1550114696216227844,en
63,𝐖𝐄𝐋𝐓𝐌𝐄𝐈𝐒𝐓𝐄𝐑 🏆🏆🏆🏆🏆 onthisday became king world ...,2022-07-21 13:45:05+00:00,1550114644076793862,en
64,one greatest seasons f1 history 🐐 essereferrari 🔴,2022-07-21 13:45:01+00:00,1550114628243312641,en


In [18]:
#import the library
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#calculate the negative, positive, neutral and compound scores, plus verbal evaluation
def sentiment_vader(row):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(row['text'])
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"

    elif sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"

    else :
        overall_sentiment = "Neutral"
  
    return negative, neutral, positive, compound, overall_sentiment

In [47]:
df["negative"], df["neutral"], df["positive"], df["compound"], df["overall_sentiment"] = zip(*df.apply(sentiment_vader, axis=1))